In [6]:
import xml.etree.ElementTree as ET
import re
from nltk.corpus import stopwords
import pandas as pd

In [7]:
# read the latest mass production table
# for 6.05 clean
infile_table = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
outfile_folder = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 clean
# infile_table = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
# infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# outfile_folder = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

# for 6.05 translate
# infile_table = "./Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt"
# infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# outfile_folder = "./Output_translated_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 translate
# infile_table = "./Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt"
# infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# outfile_folder = "./Output_translated_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

In [8]:
table_new = pd.read_csv(infile_table, sep="\t", index_col=0, keep_default_na=False)

In [9]:
# replace variable with value in the latest mass production table
# iterate column and row in table_new
for column in range(table_new.shape[1]):
    for row in range(table_new.shape[0]):
        content_new = str(table_new.iloc[row, column])
        start_pattern = "%\("
        # count the number of the replacement in one variable(content_new)
        count = [match.start() for match in re.finditer(start_pattern, str(content_new))]
        for _ in range(len(count)):
            # find the variable
            start_pattern = "%\("
            start_index = [match.start() for match in re.finditer(start_pattern, str(content_new))]
            end_pattern = "\)%"
            end_index = [match.start() for match in re.finditer(end_pattern, str(content_new))]
            # no need for replacement
            if start_index == []:
                continue
            else:
                variable = content_new[start_index[0]: end_index[0]+2]
                # find the corresponding column name, and then find the value
                column_name = table_new.columns[column]
                try:
                    value = table_new.loc[variable, column_name] 
                    # print(variable, value)
                    content_new = content_new.replace(variable, value)
                    table_new.iloc[row, column] = content_new
                except:
                    print(variable + " doesn't exist")
# table_new.head()
for i in range(len(table_new.columns)):
    column_name = table_new.columns[i]
    # print(table_new.columns[i])
    fout = outfile_folder + str(table_new.columns[i]) + ".brd"
    # print(fout)
    count_line = 0
    count_text = 0
    with open(infile_brd, 'r') as infile, open(fout, 'w+') as outfile:
        for line in infile:
            line = line.replace('\r', '')
            line_str = str(line)
            # replace massproduce
            start_pattern_problem_name = "<ProblemName>"
            start_index_problem_name = [match.start() for match in re.finditer(start_pattern_problem_name, line_str)]
            end_pattern_problem_name = "</ProblemName>"
            end_index_problem_name = [match.start() for match in re.finditer(end_pattern_problem_name, line_str)]
            start_index_problem_name, end_index_problem_name
            try:
                problem_name_old = line_str[start_index_problem_name[0]+13: end_index_problem_name[0]]
                line_str = line_str.replace(problem_name_old, column_name)
            except:
                pass
            # replace text in first node
            if count_text == 0:
                start_pattern_first_node = '<text>'
                start_index_first_node = [match.start() for match in re.finditer(start_pattern_first_node, line_str)]
                end_pattern_first_node = '</text>'
                end_index_first_node = [match.start() for match in re.finditer(end_pattern_first_node, line_str)]
                if start_index_first_node != []:
                    # print(start_index_first_node, count_line)
                    count_text += 1
                    # print(line_str[start_index_first_node[0]+6: end_index_first_node[0]])
                    text_old = line_str[start_index_first_node[0]+6: end_index_first_node[0]]
                    line_str = line_str.replace(text_old, column_name)
            # count the number of the replacement in one variable(line_str)
            start_pattern = "%\("
            count = [match.start() for match in re.finditer(start_pattern, str(line_str))]
            if count == []:
                line_str = line_str
            else:
                for _ in range(len(count)):
                    start_pattern = "%\("
                    start_index = [match.start() for match in re.finditer(start_pattern, str(line_str))]
                    end_pattern = "\)%"
                    end_index = [match.start() for match in re.finditer(end_pattern, str(line_str))]
                    if start_index == []:
                        line_str = line_str
                    else:
                        variable = line_str[start_index[0]: end_index[0]+2]
                    try:
                        value = table_new.loc[variable, column_name] 
                        # print(variable, value)
                        line_str = line_str.replace(variable, value).replace("<%", "&lt;%").replace("%>", "%&gt;")
                    except:
                        print(variable + " doesn't exist")
            count_line += 1
            # print(str(count_line) + line_str)
            outfile.write(line_str)
        print(fout.split("/")[-1] + " finished")

%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
1.brd finished
2.brd finished
3.brd finished
4.brd finished
5.brd finished
6.brd finished
7.brd finished
8.brd finished
